In [1]:
!nvidia-smi

Thu Apr 27 21:57:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:15:00.0 Off |                    0 |
| N/A   33C    P0    55W / 300W |  13129MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import numpy as np
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision as tv
from torch.utils.data.dataloader import DataLoader
from torchvision import models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import ipdb

In [3]:
class SimpsonsTestset(Dataset):
    def __init__(self, folder=None):
        super().__init__()
        
        self.testset = os.listdir(folder)
        self.folder = self.testset.copy()
        for i in range(len(self.testset)):
            self.testset[i] = tv.io.read_image(folder + '/' + self.testset[i])
            self.testset[i] = self.testset[i].resize_(3, 156, 156)
            self.testset[i] = self.testset[i].to(torch.float32)
            self.testset[i] = self.testset[i].to(device) / 255
        
        self.dictionary = tv.datasets.ImageFolder('/usr/src/app/simpsons_dataset_kaggle')
        self.dictionary = self.dictionary.class_to_idx
        self.dictionary = dict((v,k) for k,v in self.dictionary.items())
    
    def __getitem__(self, index):
        return self.testset[index], self.folder[index]
    
    def __len__(self):
        return len(self.testset)

In [4]:
def test_step():
    model.eval()

    correct_total = 0
    k = 0
    with torch.no_grad():
        for images, filenames in test_loader.dataset:
            images = images.to(device)
            
            output = model(images.unsqueeze(0))
            
            prediction = test_loader.dataset.dictionary[output.argmax(dim=1).item()]
            if filenames.find(prediction) != -1:
                correct_total += 1
                
            k += 1
            
            #correct_total += prediction.eq(labels.view_as(prediction)).sum()
            
            #loss = criterion(output, labels)
            #running_loss += loss
    
    return correct_total

def load_model():
    return torch.load('best_accuracy_point')

In [5]:
model = load_model()
trans=tv.transforms.Compose([tv.transforms.Resize([156,156]),
                             tv.transforms.ToTensor(),
                             tv.transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255])])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

folder = 'kaggle_simpson_testset'
testset = SimpsonsTestset(folder=folder)

test_loader = DataLoader(dataset=testset, batch_size=1, shuffle=False)
model.to(device)

print('{0}/{1} classified correctly'.format(test_step(), len(testset)))

cuda:0
50/990 classified correctly
